# Crawling

In [1]:
# Import required Python package
!pip install pandas

# Install Node.js (because tweet-harvest built using Node.js)
!sudo apt-get update
!sudo apt-get install -y ca-certificates curl gnupg
!sudo mkdir -p /etc/apt/keyrings
!curl -fsSL https://deb.nodesource.com/gpgkey/nodesource-repo.gpg.key | sudo gpg --dearmor -o /etc/apt/keyrings/nodesource.gpg

!NODE_MAJOR=20 && echo "deb [signed-by=/etc/apt/keyrings/nodesource.gpg] https://deb.nodesource.com/node_$NODE_MAJOR.x nodistro main" | sudo tee /etc/apt/sources.list.d/nodesource.list

!sudo apt-get update
!sudo apt-get install nodejs -y

!node -v

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,057 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,617 kB]
Get:13 http://

In [8]:
# Crawl Data

filename = 'Crawl_Twit_pilpres_2024.csv'
search_keyword = 'Pilpres 2024 AND Milih OR dukung OR pilih AND Ganjar OR Anies OR Prabowo OR Paslon OR Calon OR Presiden OR Wakil Presiden since:2022-10-01'
limit = 300

!npx --yes tweet-harvest@2.2.8 -o "{filename}" -s "{search_keyword}" -l {limit} --token ""


Welcome to the Twitter Crawler 🕷️

This script uses Chromium Browser to crawl data from Twitter with *your* Twitter auth token.
Please enter your Twitter auth token when prompted.

Note: Keep your access token secret! Don't share it with anyone else.
Note: This script only runs on your local device.

? What's your Twitter auth token? › 78? What's your Twitter auth token? › *78? What's your Twitter auth token? › **78? What's your Twitter auth token? › ***78? What's your Twitter auth token? › ****78? What's your Twitter auth token? › *****78? What's your Twitter auth token? › ******78? What's your Twitter auth token? › *******78? What's your Twitter auth token? › ********78? What's your Twitter auth token? › *********78? What's your Twitter auth token? › **********78? What's your Twitter auth token? › ***********78? What's your Twitter auth token? › ************78? What's your Twitter auth token? › *************78? What's your Twitter auth token? › **********

In [ ]:
import pandas as pd

# Specify the path to your CSV file
file_path = f"tweets-data/{filename}"

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
display(df)

In [31]:
df1 = pd.DataFrame(df['full_text'])
df1

,full_text
0,BERITA SEMASA JANUARI Kepentingan dan kebaj...
1,narkosun Pilpres tahun kita harus pilih presi...
2,Pandji Pragiwaksono mengaku belum menentukan a...
3,Maruarar Sirait Saya Dukung PrabowoGibran di P...
4,Ayo semua nya kita dukung untuk terus menjaga...
...,...
278,tribunnews Pilpres tahun pilih presiden yang ...
279,okezonenews Pilpres tahun kita harus pilih pr...
280,kompascom Pilpres tahun pilih presiden yang m...
281,okezonenews Pilpres tahun pilih presiden yang...


In [33]:
df1.to_csv('pilpres2024_text.csv', index=False)

# Cleaning

In [15]:
import pandas as pd
import re

In [34]:
df = pd.read_csv('pilpres2024_text.csv')

In [35]:
# Kolom yang akan dibersihkan
kolom_teks = 'full_text'

# Fungsi untuk membersihkan teks dari punctuation, angka, dan karakter tidak penting
def bersihkan_teks(full_text):
    full_text = re.sub(r'http\S+', '', str(full_text))
    teks_bersih = re.sub(r'[^a-zA-Z\s]', '', str(full_text))
    return teks_bersih

# Terapkan fungsi pembersihan pada kolom
df[kolom_teks] = df[kolom_teks].apply(bersihkan_teks)

In [36]:
df.to_csv('pilpres2024_clean.csv', index=False)

# Labeling

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [38]:
df = pd.read_csv('pilpres2024_clean.csv')
df

,full_text
0,BERITA SEMASA JANUARI Kepentingan dan kebaj...
1,narkosun Pilpres tahun kita harus pilih presi...
2,Pandji Pragiwaksono mengaku belum menentukan a...
3,Maruarar Sirait Saya Dukung PrabowoGibran di P...
4,Ayo semua nya kita dukung untuk terus menjaga...
...,...
278,tribunnews Pilpres tahun pilih presiden yang ...
279,okezonenews Pilpres tahun kita harus pilih pr...
280,kompascom Pilpres tahun pilih presiden yang m...
281,okezonenews Pilpres tahun pilih presiden yang...


In [44]:
import pandas as pd
import re

# Daftar kata kunci dan pola untuk setiap kategori
kata_kunci = {
    'Paslon_1': ['pilih Anies',  'Pilih no satu', 'Pilih Nomer satu', 'pilih nomer satu','pilih Muhaimin', 'pilih AMIN','bersama AniesMuhaimin','menangkan AniesMuhaimin',
                 'dukung paslon AniesMuhaimin','dukung paslon Anies', 'dukung amin', 'bersama anies', 'menangkan anies', 'dukung Anies', 'dukung Muhaimin', 'dukung Anies Muhaimin',
                 'dukung AMIN', 'Anies Muhaimin pilih mereka', 'pilih Calon Presiden RI Anies'],
    'Paslon_2': ['pilih Prabowo', 'Pilih No dua', 'Pilih nomer dua', 'Pilih nomor dua', 'pilih Gibran', 'pilih Prabowo Gibran','bersama PrabowoGibran',
                 'menangkan PrabowoGibran','menangkan prabowo', 'dukung paslon PrabowoGibran', 'Dukung Pak Prabowo', 'Dukung Mas Gibran', 'dukung Prabowo', 'dukung Gibran',
                 'dukung Prabowo Gibran', 'Prabowo Gibran pilih mereka', 'pilih Calon Presiden RI prabowo'],
    'Paslon_3': ['pilih Ganjar', 'Pilih no tiga', 'Pilih Nomer tiga', 'pilih nomer tiga', 'pilih Mahfud', 'pilih Ganjar Mahfud','bersama GanjarMahfud',
                 'menangkan GanjarMahfud', 'Menangkan Ganjar', 'Menangkan Mahfud','dukung paslon GanjarMahfud', 'dukung Ganjar', 'dukung Mahfud',
                 'dukung Ganjar Mahfud','Ganjar Mahfud pilih mereka', 'pilih Calon Presiden RI Ganjar'],
}

# Fungsi untuk label otomatis berdasarkan kata kunci
def label_otomatis(teks, kata_kunci):
    teks_lower = teks.lower()

    for label, pola_kata_kunci in kata_kunci.items():
        for kata_kunci in pola_kata_kunci:
            if re.search(kata_kunci, teks_lower, flags=re.IGNORECASE):
                return label

    return 'Tidak Diketahui'

# Membaca file CSV
file_path = 'pilpres2024_clean.csv'
df = pd.read_csv(file_path)

# Menambahkan kolom 'Label' berdasarkan label otomatis
df['Label'] = df['full_text'].apply(label_otomatis, kata_kunci=kata_kunci)

# Menyimpan DataFrame yang sudah diberi label ke file CSV baru
df.to_csv('pilpres2024_labeled.csv', index=False)

# Menampilkan beberapa baris pertama dari DataFrame yang sudah diberi label
print(df.head())


                                           full_text            Label
0  BERITA SEMASA  JANUARI   Kepentingan dan kebaj...  Tidak Diketahui
1  narkosun Pilpres tahun  kita harus pilih presi...  Tidak Diketahui
2  Pandji Pragiwaksono mengaku belum menentukan a...  Tidak Diketahui
3  Maruarar Sirait Saya Dukung PrabowoGibran di P...         Paslon_2
4  Ayo  semua nya kita dukung untuk terus menjaga...  Tidak Diketahui


# Analisis Sentimen

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [46]:
df = pd.read_csv('pilpres2024_labeled.csv')
df

,full_text,Label
0,BERITA SEMASA JANUARI Kepentingan dan kebaj...,Tidak Diketahui
1,narkosun Pilpres tahun kita harus pilih presi...,Tidak Diketahui
2,Pandji Pragiwaksono mengaku belum menentukan a...,Tidak Diketahui
3,Maruarar Sirait Saya Dukung PrabowoGibran di P...,Paslon_2
4,Ayo semua nya kita dukung untuk terus menjaga...,Tidak Diketahui
...,...,...
278,tribunnews Pilpres tahun pilih presiden yang ...,Tidak Diketahui
279,okezonenews Pilpres tahun kita harus pilih pr...,Tidak Diketahui
280,kompascom Pilpres tahun pilih presiden yang m...,Tidak Diketahui
281,okezonenews Pilpres tahun pilih presiden yang...,Tidak Diketahui


In [47]:
# Pemrosesan data
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['full_text'])
y = df['Label']

# Bagi data menjadi data latih dan uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi dan pelatihan model SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Prediksi pada data uji
y_pred = svm_model.predict(X_test)

# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred, zero_division=1))

Accuracy: 0.9473684210526315
                 precision    recall  f1-score   support

       Paslon_1       1.00      0.00      0.00         1
       Paslon_2       1.00      1.00      1.00         1
       Paslon_3       1.00      0.94      0.97        34
Tidak Diketahui       0.88      1.00      0.93        21

       accuracy                           0.95        57
      macro avg       0.97      0.74      0.73        57
   weighted avg       0.95      0.95      0.94        57

